<a href="https://colab.research.google.com/github/vnavya2004/BTP/blob/main/Paper(1_s2_0_S0957417422020255_main)_English.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1: Install Required Libraries
!pip install numpy pandas tensorflow keras gensim nltk scikit-learn openpyxl


In [2]:

# Step 2: Import Libraries
import numpy as np
import pandas as pd
import gensim
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Conv1D, MaxPooling1D, Dense, Dropout, Input, Bidirectional, Attention, GlobalAveragePooling1D, Concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import os

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:


# Step 3: Load and Preprocess Data
# Load the Excel file
from google.colab import files
uploaded = files.upload()  # Manually upload your .xlsx file

# Change the file name as per the uploaded file
df = pd.read_excel(next(iter(uploaded.keys())))  # Replace with 'filename.xlsx' if needed


Saving newenglish.xlsx to newenglish.xlsx


In [4]:
df = df.sample(frac=0.4, random_state=42)
print(df.head())

                                                  tweets  labels
7845   stressed about movingfinding apartment think a...       1
44336  wonder priority goodnim glad waited that could...       0
43452                             glad love them xfxfxxd       0
58975  ixexxve never spoken publicly about marriage b...       0
16152               sexy bitches driving anxiety xfxfxxc       1


In [5]:

# Define the text and label columns
text_column_name = 'tweets'  # replace with actual column name for text
label_column_name = 'labels'  # replace with actual column name for labels


In [6]:

# Preprocess text data
def preprocess_text(text):
    tokens = word_tokenize(str(text).lower())
    words = [word for word in tokens if word.isalpha()]
    words = [word for word in words if word not in stopwords.words('english')]
    return " ".join(words)

df[text_column_name] = df[text_column_name].apply(preprocess_text)

# Step 4: Split Data into Train and Test Sets
X_train, X_test, y_train, y_test = train_test_split(df[text_column_name], df[label_column_name], test_size=0.2, random_state=42)


In [9]:
!wget -c "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz"
!gunzip cc.en.300.vec.gz
fasttext_model = gensim.models.KeyedVectors.load_word2vec_format('cc.en.300.vec')


--2024-10-28 05:41:37--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.226.210.15, 13.226.210.111, 13.226.210.25, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.226.210.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1325960915 (1.2G) [binary/octet-stream]
Saving to: ‘cc.en.300.vec.gz’

cc.en.300.vec.gz    100%[===================>]   1.23G   176MB/s    in 14s     

2024-10-28 05:41:52 (89.4 MB/s) - ‘cc.en.300.vec.gz’ saved [1325960915/1325960915]



In [10]:
from google.colab import drive
drive.mount('/content/drive')
# Load FastText embeddings
# Path to save the model in Google Drive
save_path = '/content/drive/MyDrive/embeddings/cc.en.300.model'

Mounted at /content/drive


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/embeddings/cc.en.300.model.vectors.npy'

In [11]:
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# Now save the model
fasttext_model.save(save_path)
# fasttext_model = gensim.models.KeyedVectors.load('/content/drive/MyDrive/embeddings/cc.en.300.model')

In [12]:
# GloVe
!wget -c "http://nlp.stanford.edu/data/glove.6B.zip"
!unzip glove.6B.zip
glove_model = gensim.models.KeyedVectors.load_word2vec_format('glove.6B.300d.txt', binary=False, no_header=True)

save_path = '/content/drive/MyDrive/embeddings/glove.6B.300d.txt'
os.makedirs(os.path.dirname(save_path), exist_ok=True)
glove_model.save(save_path)
# glove_model = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/embeddings/glove.6B.300d.txt', binary=False, no_header=True)


--2024-10-28 05:55:15--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-10-28 05:55:15--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-10-28 05:55:15--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [14]:

# save_path = '/content/drive/MyDrive/embeddings/GoogleNews-vectors-negative300.bin'
# os.makedirs(os.path.dirname(save_path), exist_ok=True)
# glove_model.save(save_path)
word2vec_model=gensim.models.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/embeddings/GoogleNews-vectors-negative300.bin', binary=False, no_header=True)


KeyboardInterrupt: 

In [ ]:


def get_embedding_matrix(word_index, embedding_model, embedding_dim=300):
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        if word in embedding_model:
            embedding_matrix[i] = embedding_model[word]
    return embedding_matrix

# Step 6: Prepare Input Data
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(X_train)

train_sequences = tokenizer.texts_to_sequences(X_train)
test_sequences = tokenizer.texts_to_sequences(X_test)

padded_train = pad_sequences(train_sequences, maxlen=100)
padded_test = pad_sequences(test_sequences, maxlen=100)
word_index = tokenizer.word_index

# Step 7: Build the Hybrid Model
embedding_dim = 300

# Input layer
input_layer = Input(shape=(100,))

# Embedding layers for each embedding type
embedding_layer_w2v = Embedding(len(word_index) + 1, embedding_dim, weights=[get_embedding_matrix(word_index, word2vec_model)], trainable=False)(input_layer)
embedding_layer_ft = Embedding(len(word_index) + 1, embedding_dim, weights=[get_embedding_matrix(word_index, fasttext_model)], trainable=False)(input_layer)
embedding_layer_glove = Embedding(len(word_index) + 1, embedding_dim, weights=[get_embedding_matrix(word_index, glove_model)], trainable=False)(input_layer)

# Concatenate embeddings
concatenated_embeddings = Concatenate()([embedding_layer_w2v, embedding_layer_ft, embedding_layer_glove])


In [23]:
from keras.models import Model
from keras.layers import Input, Bidirectional, LSTM, Conv1D, GlobalMaxPooling1D, Dense, Dropout, Attention
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

# Assuming `input_layer` and `concatenated_embeddings` are already defined
# Layer definitions
bilstm_out = Bidirectional(LSTM(100, return_sequences=True))(concatenated_embeddings)
lstm_out = LSTM(100, return_sequences=True)(bilstm_out)
cnn_out = Conv1D(filters=50, kernel_size=3, activation='relu')(lstm_out)

attention_out = Attention()([cnn_out, cnn_out])  # Adjusted for attention layer

# Global max pooling and dense layers
max_pool_out = GlobalMaxPooling1D()(attention_out)  # Applying GlobalMaxPooling1D after attention
dropout_layer_1 = Dropout(0.5)(max_pool_out)
dense_layer_1 = Dense(250, activation='relu')(dropout_layer_1)
dropout_layer_2 = Dropout(0.5)(dense_layer_1)
output_layer = Dense(1, activation='sigmoid')(dropout_layer_2)  # Use sigmoid for binary classification

# Define and compile model
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  # Use binary crossentropy for binary classification
model.summary()

# Step 8: Train the Model
model.fit(padded_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

# Step 9: Evaluate the Model
predictions = model.predict(padded_test)
predicted_classes = (predictions > 0.5).astype("int32")  # Get the predicted class indices based on a threshold
print("Accuracy:", accuracy_score(y_test, predicted_classes))
print("AUC:", roc_auc_score(y_test, predictions))  # For binary classification, use the predictions directly
print("Classification Report:\n", classification_report(y_test, predicted_classes))


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 100)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_2 (Embedding)   │ (None, 100, 300)       │      9,930,900 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_3 (Embedding)   │ (None, 100, 300)       │      9,930,900 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_1             │ (None, 100, 600)       │              0 │ embedding_2[0][0],     │
│ (Concatenate)             │                        │                │ embedding_3[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_7           │ (None, 100, 200)       │        560,800 │ concatenate_1[0][0]    │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_13 (LSTM)            │ (None, 100, 100)       │        120,400 │ bidirectional_7[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_7 (Conv1D)         │ (None, 98, 50)         │         15,050 │ lstm_13[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention_7 (Attention)   │ (None, 98, 50)         │              0 │ conv1d_7[0][0],        │
│                           │                        │                │ conv1d_7[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_5    │ (None, 50)             │              0 │ attention_7[0][0]      │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_12 (Dropout)      │ (None, 50)             │              0 │ global_max_pooling1d_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_12 (Dense)          │ (None, 250)            │         12,750 │ dropout_12[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_13 (Dropout)      │ (None, 250)            │              0 │ dense_12[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_13 (Dense)          │ (None, 1)              │            251 │ dropout_13[0][0]       │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 20,571,051 (78.47 MB)

 Trainable params: 709,251 (2.71 MB)

 Non-trainable params: 19,861,800 (75.77 MB)

Epoch 1/10
542/542 ━━━━━━━━━━━━━━━━━━━━ 19s 23ms/step - accuracy: 0.8244 - loss: 0.3940 - val_accuracy: 0.8998 - val_loss: 0.2554
Epoch 2/10
542/542 ━━━━━━━━━━━━━━━━━━━━ 23s 27ms/step - accuracy: 0.8972 - loss: 0.2509 - val_accuracy: 0.9055 - val_loss: 0.2552
Epoch 3/10
542/542 ━━━━━━━━━━━━━━━━━━━━ 16s 20ms/step - accuracy: 0.9172 - loss: 0.2096 - val_accuracy: 0.9237 - val_loss: 0.2057
Epoch 4/10
542/542 ━━━━━━━━━━━━━━━━━━━━ 24s 27ms/step - accuracy: 0.9334 - loss: 0.1683 - val_accuracy: 0.9195 - val_loss: 0.2053
Epoch 5/10
542/542 ━━━━━━━━━━━━━━━━━━━━ 11s 21ms/step - accuracy: 0.9461 - loss: 0.1334 - val_accuracy: 0.9252 - val_loss: 0.2024
Epoch 6/10
542/542 ━━━━━━━━━━━━━━━━━━━━ 13s 24ms/step - accuracy: 0.9615 - loss: 0.1009 - val_accuracy: 0.9232 - val_loss: 0.2308
Epoch 7/10
542/542 ━━━━━━━━━━━━━━━━━━━━ 21s 25ms/step - accuracy: 0.9709 - loss: 0.0773 - val_accuracy: 0.9283 - val_loss: 0.2456
Epoch 8/10
542/542 ━━━━━━━━━━━━━━━━━━━━ 21s 25ms/step - accuracy: 0.9784 - loss: 0.0558 - 